In [1]:
import pandas as pd

import pickle
from pathlib import Path

Make path of monthly directories

In [27]:
all_months = ["2023-05-01_2023-05-31", "2023-06-01_2023-06-30", "2023-07-01_2023-07-31", "2023-08-01_2023-08-31", "2023-09-01_2023-09-30",
    "2023-10-01_2023-10-31", "2023-11-01_2023-11-30", "2023-12-01_2023-12-31", "2024-01-01_2024-01-31", "2024-02-01_2024-02-29",
    "2024-03-01_2024-03-31", "2024-04-01_2024-04-30"]

base_img_dir = Path("/projects/frame_align/data/img_data/")
month_dir_paths = [base_img_dir / month for month in all_months]
sorted_month_dir_paths = sorted(month_dir_paths)

print("month_dir_paths", sorted_month_dir_paths)

month_dir_paths [PosixPath('/projects/frame_align/data/img_data/2023-05-01_2023-05-31'), PosixPath('/projects/frame_align/data/img_data/2023-06-01_2023-06-30'), PosixPath('/projects/frame_align/data/img_data/2023-07-01_2023-07-31'), PosixPath('/projects/frame_align/data/img_data/2023-08-01_2023-08-31'), PosixPath('/projects/frame_align/data/img_data/2023-09-01_2023-09-30'), PosixPath('/projects/frame_align/data/img_data/2023-10-01_2023-10-31'), PosixPath('/projects/frame_align/data/img_data/2023-11-01_2023-11-30'), PosixPath('/projects/frame_align/data/img_data/2023-12-01_2023-12-31'), PosixPath('/projects/frame_align/data/img_data/2024-01-01_2024-01-31'), PosixPath('/projects/frame_align/data/img_data/2024-02-01_2024-02-29'), PosixPath('/projects/frame_align/data/img_data/2024-03-01_2024-03-31'), PosixPath('/projects/frame_align/data/img_data/2024-04-01_2024-04-30')]


Count number of jpg files in each directory

In [29]:
from tqdm import tqdm

count_monthly_img = {}
for idx, month_dir in enumerate(sorted_month_dir_paths):
    dir_path = month_dir / "correct_downloaded_imgs"
    # count the number of images in the month directory and its subdirectories
    count_monthly_img[month_dir.name] = len(list(dir_path.glob('*.jpg')))
    print(f"{month_dir.name}: {count_monthly_img[month_dir.name]}")


2023-05-01_2023-05-31: 55647
2023-06-01_2023-06-30: 24812
2023-07-01_2023-07-31: 31114
2023-08-01_2023-08-31: 40424
2023-09-01_2023-09-30: 40203
2023-10-01_2023-10-31: 36545
2023-11-01_2023-11-30: 38530
2023-12-01_2023-12-31: 34856
2024-01-01_2024-01-31: 40821
2024-02-01_2024-02-29: 37074
2024-03-01_2024-03-31: 34942
2024-04-01_2024-04-30: 34864


Make pickle file of the count of jpg files in each directory

In [ ]:
# make pickle file which is dict of month and uuids in sorted_month_dir_paths/correct_downloaded_imgs

for idx, month_dir in enumerate(tqdm(sorted_month_dir_paths)):
    dir_path = month_dir / "correct_downloaded_imgs"
    uuids = [img_path.stem for img_path in dir_path.glob("*.jpg")]
    # print len of duplicated uuids
    print(f"len of duplicated uuids: {len(uuids) - len(set(uuids))}")
    month = month_dir.name
    pickle_path = month_dir / f"{month}_downloaded_uuids.pkl"
    with open(pickle_path, "wb") as f:
        pickle.dump(uuids, f)

    print(f"pickle_path: {pickle_path}")   

[Used for Verification] Load pickle file and check the count of jpg files in each directory

In [32]:
# load and count all pickle files
count_monthly_img = {}
for idx, month_dir in enumerate(sorted_month_dir_paths):
    with open(month_dir / f"{month_dir.name}_downloaded_uuids.pkl", "rb") as f:
        img_names = pickle.load(f)
        count_monthly_img[month_dir.name] = len(img_names)
        print(f"{month_dir.name}: {count_monthly_img[month_dir.name]}")

2023-05-01_2023-05-31: 55647
2023-06-01_2023-06-30: 24812
2023-07-01_2023-07-31: 31114
2023-08-01_2023-08-31: 40424
2023-09-01_2023-09-30: 40203
2023-10-01_2023-10-31: 36545
2023-11-01_2023-11-30: 38530
2023-12-01_2023-12-31: 34856
2024-01-01_2024-01-31: 40821
2024-02-01_2024-02-29: 37074
2024-03-01_2024-03-31: 34942
2024-04-01_2024-04-30: 34864


In [ ]:
raw_month_dir = Path("/projects/frame_align/data/raw/2023-2024")
img_dir = month_raw_dir / "img_data/"

In [3]:
file_sizes = {}
for img_file in img_dir.glob("*.jpg"):
    file_sizes[img_file.name.split(".")[0]] = img_file.stat().st_size
file_size_df = pd.DataFrame.from_dict(file_sizes, orient='index', columns=['file_size'])

In [4]:
file_size_df = file_size_df[file_size_df['file_size'] <= file_size_df['file_size'].quantile(0.95)]
file_size_df = file_size_df[file_size_df['file_size'] >= 5000]
# file_size_df.sort_values('file_size', ascending=True).hist(bins=5)

In [5]:
img_uuids = list(file_size_df.index)

In [6]:
len(img_uuids)

36601

In [7]:
text_df = pd.read_csv(month_dir/"datawithtopiclabels.csv")

In [8]:
text_df.columns

Index(['topic', 'auto_topic_label', 'id', 'authors', 'date_download',
       'date_modify', 'date_publish', 'description', 'filename', 'image_url',
       'language', 'localpath', 'maintext', 'source_domain', 'title',
       'title_page', 'title_rss', 'url', 'month', 'topic_label'],
      dtype='object')

In [9]:
text_df.dropna(subset=['maintext'], inplace=True)
text_df['text_length'] = text_df['maintext'].apply(lambda x: len(x))

In [10]:
lower_quantile = text_df['text_length'].quantile(0.05)
upper_quantile = text_df['text_length'].quantile(0.95)

text_df = text_df[text_df['text_length'] >= lower_quantile]
text_df = text_df[text_df['text_length'] <= upper_quantile]

In [17]:
filtered_text_df = []
for topic_label, topic_df in text_df.groupby('topic_label'):
    if len(topic_df) > 5000:
        topic_df = topic_df.sample(5000)
    filtered_text_df.append(topic_df)
filtered_text_df = pd.concat(filtered_text_df)

In [19]:
len(filtered_text_df)

17084

In [20]:
text_uuids = list(filtered_text_df.id)

In [21]:
common_uuids = set(text_uuids).intersection(set(img_uuids))

In [22]:
len(common_uuids)

16140

In [23]:
pickle.dump(common_uuids, open(month_dir/"selected_uuids.pkl", "wb"))